before starting with the data enrichment process, we need to do some DATA EXPLORATION to understand the data.

#DATA EXPLORATION STAGE:

#CALLING DATA MUNGING NOTEBOOK

we are calling DATA MUNGING notebook here using %run command.

In [0]:
%run "/Workspace/Users/dhivyadharshini298@gmail.com/databricks-code-repository/Bread_and_Butter_Transformation_Analytical_operations_DSL/1.DATA_MUNGING"

In [0]:
dedup_df3.printSchema()

In [0]:
print("Records got cleaned/munged ",dfmethod1.count()-len(dedup_df3.collect()))
display(dedup_df3.summary())
display(dedup_df3)

#DATA ENRICHMENT STAGE:

**Best practices of using different column enrichment functions**
1. select is good to use if we want to perform - Good for ordering/reordering, only renaming column (not good), only reformatting/deriving a column (not good), for all of these operation in a single iteration such renaming, reordering, reformatting,deriving, dropping etc., (best to use)
2. selectExpr is good to use if we want to perform - Same as select by using iso sql functionality (if we are not familiar in DSL) for all of these operation in a single iteration
3. withColumn is good to use if we want to perform - for adding/deriving/modifying/replacing in a single iteration Adding/Deriving column(s) in the last (Good), Modifying/replacing (Good), Renaming (not good), Dropping(not possible)
4. withColumnRenamed is good to use if we want to perform - only for renaming column (Good)
5. drop is good to use if we want to perform - only dropping of columns in the given position (Good)

##A.Add (withColumn,select,selectExpr), Derive (withColumn,select,selectExpr), Rename (withColumnRenamed,select,selectExpr), Modify/replace (withColumn, select/selectExpr), Remove/Eliminate (drop,select,selectExpr) - (very important spark sql functions)

###1.Adding of columns

In [0]:
datadt="25-12-25"
enrich_df1=dedup_df3.withColumn("loaddt",current_date()).withColumn("datadt",lit(datadt))
enrich_df1.show(2)
#or

In [0]:
enrich_df1=dedup_df3.withColumns({"loaddt":current_date(),"datadt":lit(datadt)})
enrich_df1.show(2)
#or

In [0]:
#pure DSL
enrich_df1=dedup_df3.select("*",current_date().alias("loaddt"),lit(datadt).alias("datadt"))
enrich_df1.show(2)


In [0]:
#or
#Only select Function that supports both DSL and SQL
datadt="25-12-25"
enrich_df1=dedup_df3.selectExpr("*","current_date() as loaddt",f"'{datadt}' as datadt")
enrich_df1.show(2)

In [0]:
#Adding an existing column in a new column (copying a column value to a new column)
#enrich_df1=dedup_df3.withColumn("sourcesystem1",col("sourcesystem"))
#enrich_df1.show(2)

###2.Deriving of columns

In [0]:
enrich_df2=enrich_df1.withColumn("profession_flag",substring("profession",1,1))
enrich_df2.show(10)

In [0]:
#or we can achieve using select
enrich_df2=enrich_df1.select("*",substring("profession",1,1).alias("profession_flag"))
enrich_df2.show(10)

In [0]:
#or we can achieve using selectExpr
enrich_df2=enrich_df1.selectExpr("*","substr(profession,1,1) as profession_flag")
enrich_df2.show(10)

###3.Renaming of columns

In [0]:
enrich_df3=enrich_df2.withColumnRenamed("profession_flag","proflag")#better to use
enrich_df3.show(10)
enrich_df3=enrich_df2.withColumnsRenamed({"profession_flag":"proflag","profession":"prof"})
enrich_df3.show(10)

In [0]:
#or
enrich_df3=enrich_df2.select("*",col("profession").alias("prof"))#This will derive a new column called prof
enrich_df3=enrich_df2.select("custid","age","firstname","lastname",col("profession").alias("prof"),"sourcesystem","loaddt","datadt",col("profession_flag").alias("proflag"))#This will derive a new column called prof#better to use
#enrich_df3=enrich_df3.drop("profession")#column orders are changed, not good to use
enrich_df3.show(10)

###4.Modify/replace (withColumn, select/selectExpr)

In [0]:
enrich_df3.printSchema()

In [0]:
enrich_df4=enrich_df3.withColumn("sourcesystem",upper(col("sourcesystem"))).withColumn("datadt",to_date(col("datadt"),'dd-MM-yy'))#This time withColumn didnt added a new column, it just modified the existing column
enrich_df4.printSchema()#datadt is expected date format
enrich_df4.show(10)

###5.Remove/Eliminate (drop,select,selectExpr)

In [0]:
enrich_df5=enrich_df4.withColumn("fullname",concat(col("firstname"),lit(" "),col("lastname")))
enrich_df5=enrich_df5.drop("firstname","lastname").select("custid","age","fullname","prof","sourcesystem","loaddt","datadt","proflag")#drop function is good to use if we just want to drop a column in the df
display(enrich_df5.limit(10))

In [0]:
#or
enrich_df5=enrich_df4.select("custid","age",concat(col("firstname"),lit(" "),col("lastname")).alias("fullname"),"prof","sourcesystem","loaddt","datadt","proflag")#Better methodology
display(enrich_df5.limit(10))

##B.Splitting & Merging/Melting of Columns

In [0]:
#Splitting of columns
enrich_df6=enrich_df5.withColumn("profsplit",split(col("prof"),' '))
enrich_df6=enrich_df6.withColumns({"proffirst":col("profsplit")[0],"proflast":col("profsplit")[size(col("profsplit"))-1]})
display(enrich_df6.limit(10))

In [0]:
#Merging of columns
enrich_df7=enrich_df6.withColumn("proflag",concat(substring(("proffirst"),1,1),substring(("proflast"),1,1))).drop("profsplit")
display(enrich_df7)

##C.Typecasting & Formatting

In [0]:
enrich_df7.printSchema()#this dataset is in an expected format for further processing

**REFERENCE CODE FOR TYPECASTING**<br>
Consider the below casting and formatting<br>
enrich_df7.printSchema()#datadt is not in a expected date format for further usage and i wanted to convert sourcesystem into uppercase<br>
enrich_df4=enrich_df3.withColumn("sourcesystem",upper(col("sourcesystem"))).withColumn("datadt",to_date(col("datadt"),'dd-MM-yy'))#This time withColumn didnt added a new column, it just modified the existing column<br>
enrich_df4.printSchema()#datadt is expected date format<br>
enrich_df4.show(10)